<a href="https://colab.research.google.com/github/NityaVerma19/Customer-Purchase-Decision-/blob/master/Consumer_behaviour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split


#Generating a dataset

### Product Info

In [119]:
#Green Packaging - 0/1


def generate_random_series(length, seed = 42):
    random.seed(seed
    )
    return [random.choice([0, 1]) for _ in range(length)]

length = 500
green_pack = generate_random_series(length, seed = 42)



In [68]:
#purchase decision - yes/no

def generate_random_series(length, seed = 42):
    random.seed(seed
    )
    return [random.choice([0, 1]) for _ in range(length)]

length = 500
purchase = generate_random_series(length, seed = 42)


In [69]:
#type of product - 'Beverage', 'Food' , 'Beauty', 'Personal hygiene', 'Paper products' , 'Clothing'


def generate_random_series(words, length, seed = 42):
    random.seed(seed)
    series = [random.choice(words) for _ in range(length)]
    return series

word_list = ['Beverage', 'Food' , 'Beauty', 'Personal hygiene', 'Paper products' , 'Clothing']
series_length = 500

product = generate_random_series(word_list, series_length, seed = 42)


In [70]:
#Price
def generate_random_series(lower_bound, upper_bound, count, seed = 42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)


lower_bound = 500
upper_bound = 5000
count = 500  # Number of random numbers to generate
price = generate_random_series(lower_bound, upper_bound, count, seed = 42)



### Customer Info

In [71]:
#Age

def generate_random_series(lower_bound, upper_bound, count, seed = 42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)


lower_bound = 18
upper_bound = 50
count = 500  # Number of random numbers to generate
age = generate_random_series(lower_bound, upper_bound, count, seed = 42)



In [72]:
#Gender


def generate_random_series(words, length, seed = 42):
    random.seed(seed)
    series = [random.choice(words) for _ in range(length)]
    return series

word_list = ['M', 'F']
series_length = 500

gender = generate_random_series(word_list, series_length, seed = 42)


In [73]:
#Income level - 15k to 150k

def generate_random_series(lower_bound, upper_bound, count, seed = 42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)


lower_bound = 15000
upper_bound = 150000
count = 500  # Number of random numbers to generate
income = generate_random_series(lower_bound, upper_bound, count, seed = 42)


In [84]:
#customer id


numbers_sequence = range(1, 500)
cust_id = pd.Series(numbers_sequence)


*Merging the series*

In [120]:
df = pd.DataFrame({'cust_id': cust_id,'product': product, 'price': price, 'green_pack': green_pack, 'age': age,'gender': gender, 'income': income, 'purchase': purchase })
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1.0,Clothing,1215,0,40,M,85238,0
1,2.0,Beverage,524,0,48,M,31089,0
2,3.0,Beverage,3695,1,43,F,139863,1
3,4.0,Clothing,2878,0,32,M,124897,0
4,5.0,Beauty,4344,0,48,M,65008,0


In [ ]:
# Downloading the dataframe

#df.to_csv('df.csv', index=False)

In [121]:
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1.0,Clothing,1215,0,40,M,85238,0
1,2.0,Beverage,524,0,48,M,31089,0
2,3.0,Beverage,3695,1,43,F,139863,1
3,4.0,Clothing,2878,0,32,M,124897,0
4,5.0,Beauty,4344,0,48,M,65008,0


In [122]:
df.shape

(500, 8)


---





# Logistic Regression


In [12]:
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1,Clothing,3961,Yes,48,M,28863,0
1,2,Beverage,3849,Yes,21,M,60866,0
2,3,Beverage,4950,No,34,F,97901,1
3,4,Clothing,4964,Yes,19,M,82031,0
4,5,Beauty,2833,Yes,21,M,81903,0


In [123]:
#feature variable

X = df.drop(columns = ['cust_id', 'purchase'], axis= 1)


#target variable

y = df.purchase.values



In [124]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [125]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

((400, 6), (100, 6))

*Feature engineering*

In [126]:
# display categorical variables

categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']

categorical

['product', 'gender']

In [127]:
#display numerical variables

numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

numerical

['price', 'green_pack', 'age', 'income']

In [128]:
X_train.describe()

,price,green_pack,age,income
count,400.000000,400.000000,400.000000,400.000000
mean,2737.182500,0.505000,33.277500,83559.330000
std,1326.220283,0.500601,9.475228,39467.068947
min,502.000000,0.000000,18.000000,15687.000000
25%,1484.750000,0.000000,25.000000,50186.500000
50%,2809.000000,1.000000,33.000000,85620.000000
75%,3961.500000,1.000000,41.000000,120045.750000
max,4979.000000,1.000000,50.000000,149753.000000


In [129]:
cols = X_train.columns
cols

Index(['product', 'price', 'green_pack', 'age', 'gender', 'income'], dtype='object')

*Endoding categorical variable*

In [130]:
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['product', 'gender'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

In [131]:
X_train.head()

,product_0,product_1,product_2,price,green_pack,age,gender_0,gender_1,income
107,0,0,1,4564,0,28,0,1,118584
336,0,1,0,899,1,30,1,0,71035
71,0,1,1,805,1,19,1,0,134756
474,0,0,1,2157,0,20,0,1,77728
6,0,0,1,3022,0,26,0,1,105491


In [133]:
X_train = pd.concat([X_train[numerical], X_train[['product_0', 'product_1', 'product_2', 'gender_0', 'gender_1']]], axis=1)

X_train.head()


,price,green_pack,age,income,product_0,product_1,product_2,gender_0,gender_1
107,4564,0,28,118584,0,0,1,0,1
336,899,1,30,71035,0,1,0,1,0
71,805,1,19,134756,0,1,1,1,0
474,2157,0,20,77728,0,0,1,0,1
6,3022,0,26,105491,0,0,1,0,1


In [134]:
X_test = pd.concat([X_test[numerical], X_test[['product_0', 'product_1', 'product_2', 'gender_0', 'gender_1']]], axis=1)

X_test.head()


,price,green_pack,age,income,product_0,product_1,product_2,gender_0,gender_1
90,3101,0,25,24719,0,1,0,0,1
254,3168,0,49,133905,0,1,0,0,1
283,1282,1,18,53689,1,0,0,1,0
445,1687,0,44,141437,0,1,1,0,1
461,760,0,40,90659,0,1,0,0,1


*Feature Scaling*

In [137]:
cols = X_train.columns

In [138]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [139]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [140]:
X_test = pd.DataFrame(X_test, columns=[cols])

*Model Training*

In [146]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression


# instantiate the model
logreg = LogisticRegression(C = 0.5, solver='liblinear', random_state=0)


# fit the model
logreg.fit(X_train, y_train)

LogisticRegression(C=0.5, random_state=0, solver='liblinear')

*Predict Results*

In [147]:
y_pred_test = logreg.predict(X_test)

y_pred_test

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [148]:
# probability of getting output as 0 - no purchase

logreg.predict_proba(X_test)[:,0]

array([0.96473773, 0.97346354, 0.01826555, 0.97714442, 0.96882401,
       0.97183793, 0.02117139, 0.96589959, 0.97793556, 0.9681203 ,
       0.97182821, 0.97020815, 0.02317119, 0.02374572, 0.02456878,
       0.98011718, 0.97086444, 0.97237574, 0.97716425, 0.97701625,
       0.02575347, 0.02233662, 0.96797605, 0.02001803, 0.96711407,
       0.96929928, 0.96113545, 0.97778795, 0.97106828, 0.97330977,
       0.97123051, 0.978743  , 0.97263584, 0.97412192, 0.97045539,
       0.97494684, 0.02405244, 0.02337117, 0.9704316 , 0.97096021,
       0.97683796, 0.0219947 , 0.97510898, 0.97386096, 0.02457437,
       0.97016227, 0.96433999, 0.02367631, 0.02050568, 0.01840119,
       0.97328559, 0.97101652, 0.01813638, 0.02614673, 0.97055878,
       0.96618869, 0.0215166 , 0.97522956, 0.97841805, 0.02204865,
       0.0237381 , 0.97013196, 0.02578274, 0.97239641, 0.96848741,
       0.97211766, 0.9760302 , 0.018761  , 0.02458468, 0.97335488,
       0.97233636, 0.02090051, 0.97796513, 0.02206034, 0.97722

In [149]:
# probability of getting output as 1 - purchase

logreg.predict_proba(X_test)[:,1]

array([0.03526227, 0.02653646, 0.98173445, 0.02285558, 0.03117599,
       0.02816207, 0.97882861, 0.03410041, 0.02206444, 0.0318797 ,
       0.02817179, 0.02979185, 0.97682881, 0.97625428, 0.97543122,
       0.01988282, 0.02913556, 0.02762426, 0.02283575, 0.02298375,
       0.97424653, 0.97766338, 0.03202395, 0.97998197, 0.03288593,
       0.03070072, 0.03886455, 0.02221205, 0.02893172, 0.02669023,
       0.02876949, 0.021257  , 0.02736416, 0.02587808, 0.02954461,
       0.02505316, 0.97594756, 0.97662883, 0.0295684 , 0.02903979,
       0.02316204, 0.9780053 , 0.02489102, 0.02613904, 0.97542563,
       0.02983773, 0.03566001, 0.97632369, 0.97949432, 0.98159881,
       0.02671441, 0.02898348, 0.98186362, 0.97385327, 0.02944122,
       0.03381131, 0.9784834 , 0.02477044, 0.02158195, 0.97795135,
       0.9762619 , 0.02986804, 0.97421726, 0.02760359, 0.03151259,
       0.02788234, 0.0239698 , 0.981239  , 0.97541532, 0.02664512,
       0.02766364, 0.97909949, 0.02203487, 0.97793966, 0.02277

### Checking Accuracy

In [150]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Model accuracy score: 1.0000
